In [ ]:
# !pip install fastapi
# !pip install uvicorn
# !pip install pydantic
# !pip install scikit-learn
# !pip install requests
# !pip install pypi-json
# !pip install pyngrok
# !pip install nest-asyncio

: 

In [ ]:
!ngrok config add-authtoken 32L4iiG20IL14J4VSRrwSPHaM0v_6xEiWgMmfpmHzeHmaAX9e

In [ ]:
import pickle
import json
import uvicorn
import threading
import time
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
from pyngrok import ngrok

In [ ]:
app = FastAPI()
origins = ['*']
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

In [ ]:
class model_input(BaseModel):
    Pregnancies: int
    Glucose: int
    BloodPressure: int
    SkinThickness: int
    Insulin: int
    BMI: float
    DiabetesPedigreeFunction: float
    Age: int

In [ ]:
diabetes_model = pickle.load(open('retrained_model.sav', 'rb'))

@app.post('/diabetes_prediction')
def diabetes_pred(input_parameters: model_input):
    input_dictionary = input_parameters.dict()

    preg = input_dictionary['Pregnancies']
    glu = input_dictionary['Glucose']
    bp = input_dictionary['BloodPressure']
    skin = input_dictionary['SkinThickness']
    insulin = input_dictionary['Insulin']
    bmi = input_dictionary['BMI']
    dpf = input_dictionary['DiabetesPedigreeFunction']
    age = input_dictionary['Age']

    input_list = [preg, glu, bp, skin, insulin, bmi, dpf, age]

    prediction = diabetes_model.predict([input_list])

    if prediction[0] == 0:
        return {'prediction': 'The person is not diabetic'}
    else:
        return {'prediction': 'The person is diabetic'}

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

def run_server():
    uvicorn.run(app, port=8000, host='0.0.0.0')

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Shutting down server.")
    ngrok.kill()